In [74]:
# paramter cell do not remove!!
# nb_parm='datalake|raw/pdf|Birddiversityanddistribution|pdf||300|150'
# nb_parm='datalake|raw/text-csv|PFW_spp_translation_table_May2024|csv||300|150'
nb_parm='llmnok'
question = 'how many bird species are in migratory?'
embed_model = "mxbai-embed-large" 
gen_model = "deepseek-r1:7b"
collection = "Bridknowledge"

In [75]:
import sys
import os

sys.path.append("/home/jovyan/notebooks")
from Framework.module import Utility

In [71]:
## Do the task After this

In [77]:
print("nb_parm:", nb_parm)
print("question:", question)
print("embed_model:", embed_model)
print("gen_model:", gen_model)
print("collection:", collection)

nb_parm: llmnok
question: how many bird species are in migratory?
embed_model: mxbai-embed-large
gen_model: deepseek-r1:7b
collection: Bridknowledge


In [93]:
import bs4
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama.llms import OllamaLLM
from langchain_community.vectorstores import Weaviate
import weaviate
import pdfplumber
import io
import pandas as pd

# Retrieval 
client = Utility.registerClient()
retriever = client.collections.get(collection)
response = retriever.query.bm25( # search without model
    # query="Conclusion",
    query = "migratory",
    limit=3,
    query_properties=["content"],
    return_metadata=MetadataQuery(score=True),
)
client.close()
docs = [obj.properties['content'] for obj in response.objects]
for i in range(len(docs)):
    print(f"context {i+1}: {docs[i]}\n\n")

# Prompt
prompt = PromptTemplate.from_template(
    """
    Answer the question based only on the context.
    Context:{context}
    
    Question: {question}
    
    Answer:
    """
)

# LLM
llm = OllamaLLM(
    model="deepseek-r1:7b",
    temperature=0,
    base_url="http://host.docker.internal:11434" 
)

# Post-processing
def format_docs(docs): # result in long text type str
    # return "\n\n".join(doc.page_content for doc in docs)
    return RunnableLambda(lambda _: "\n\n".join(doc for doc in docs))

# Chain
rag_chain = (
    {"context": format_docs(docs), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
result = rag_chain.invoke(question)
print(f"\nFianl Answer: {result}")

context 1: INTRODUCTION According to Choudhury (2007), Bhutan has been fairly well covered by ornithological surveys and the entire country forms a part of Eastern Himalaya Endemic Bird area (Stattersfield et al.,1998). As per United Nations Environment Program [UNEP] and Convention on Conservation of Migratory Species [CCMS](UNEP2009), 9856 bird species are recorded worldwide. Out of which 1855 species are migratory, 262 species are seabirds, 343 species are altitudinal migrants, 181 species are nomadic and 1593 species are migratory land and waterbirds. Over 800 species of birds are estimated to be found in Bhutan of which frequent numbers of winter visitors, such as migrant thrushes are found in addition to 450 species of resident birds (Sherpa2000). Forest is the most significant habitat for birds by supporting around 75% of all bird species while only 45% of all bird species have adapted to humans modified habitats (Birdlife International2008).Human activities such as farming, set

In [61]:
# client = Utility.registerClient()
# for i in client.collections.list_all():
#     # print(client.collections.get(i))
#     print("##################################################")
#     print(i)
#     print("\n\n break line here")
# # client.collections.delete("Question")
# # client.collections.delete("Csvbrid")

                
# client.close()

# # Biodiversity in Bhutan: A Comprehensive Overview**\n\nBhutan exhibits a rich and unique biodiversity profile, as highlighted by recent studies and data. The country\'s population census conducted by the PHCB in 2005 provides valuable insights into the demographic and housing landscape, underscoring its ecological significance despite its small size.\n\n**Bird Species Diversity:**\nBhutan is renowned for its diverse bird populations, with species such as the Snow Goose (Cedrus chilinensis) and the Black-vented Weathcerbird (Parus tridactylus) being prominent. The country\'s varied habitats, including alpine meadows, forests, wetlands, and mountains, support a wide range of bird species, each adapted to specific ecological niches.\n\n**Habitat Impact:**\nThe altitudinal gradient in Bhutan supports distinct avian communities, with higher elevations hosting unique species like the Snow Goose. Urbanization and deforestation have posed challenges, affecting bird distribution patterns as noted by studies on beta diversity along environmental gradients.\n\n**Conservation Efforts:**\nBhutan has implemented measures to protect its ecological integrity, including the establishment of protected areas that support diverse avian communities. International commitments, such as the Convention on Migratory Species, reflect Bhutan\'s dedication to bird conservation and ecosystem preservation.\n\n**Ecological Methods:**\nThe application of ecological census techniques, as referenced in Gibbons and Gregory\'s work, has been instrumental in studying Bhutan\'s bird populations, emphasizing the importance of accurate data collection for biodiversity assessment.\n\nIn summary, Bhutan\'s biodiversity is a testament to its unique ecological landscape, supported by a variety of species adapted to different environments. Despite challenges from human activities, conservation efforts and international agreements ensure the preservation of this vital natural resource.'

